EPITA 2023 IML lab02_clustering_04-EM-GMM v2023-03-27_103407 by G. Tochon & J. Chazalon

<div style="overflow: auto; padding: 10px; margin: 10px 0px">
<img alt="Creative Commons License" src='img/CC-BY-4.0.png' style='float: left; margin-right: 20px'>
    
This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).
</div>

# Lab 2, part 4: Reimplement EM to train a GMM

![](img/1280px-ClusterAnalysis_Mouse.svg.png)
*Guess why it is this called the "mouse" dataset…*


In this first part, we will focus on the famous **[GMMs](https://en.wikipedia.org/wiki/Mixture_model#Gaussian_mixture_model)** which are estimated using the equally famous [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm).

Compared to K-Means, GMMs are a bit more complex to fit but they provide a much richer model of the underlying data distribution.
Indeed, GMMs represent a data distribution as a set of modes, and for each mode we estimate:

- its mean ($\mu$), alike cluster centroïds for K-Means;
- its variance ($\Sigma$) in every direction;
- its amplitude  ($\pi$).

This allows to models points clouds with anisotropic spreads, and also to perform soft assignments (i.e. compute an assignment to multiple clusters for each point).

Needless to say, the complexity of parameter estimation is highly correlated with the richness of the representation.

Please note that Gaussian Mixture Models are a particular case of Mixture models, and many variants can complexify our representation as per our needs.

Here, we will focus on re-implementing a simple EM algorithm to fit a simplied GMM.
Our code will be very unefficient, and inaccurate (in particular because we do not compute values in the log-probability space).

We will then conduct the same kind of study as for the K-Means to understand how such model can be used:

- play with scikit-learn's implementation
- understand the role of the covariance matrix
- understand how to select the number of components
- re-implement an EM algorithm to fit a GMM

# 0. Setup

## Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

## Some sample data
We suggest you work with the following set of samples points to get started.
It will enable us to illustrated in a simple way the key messages we want to share with you.

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)
transformation = [[0.6, -0.7], [-0.2, 0.9]]
X_aniso = np.dot(X, transformation)

In [ ]:
def plot_clusters(X, y=None, /, size=1, label=None):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=size, label=label)
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0)

In [ ]:
plt.figure(figsize=(16, 4))
plt.subplot(1,2,1)
plot_clusters(X_aniso,y)
plt.title("Our sample data with cluster assignment")
plt.axis("equal")
plt.subplot(1,2,2)
plot_clusters(X_aniso)
plt.title("Our sample data as our self-supervised algorithm will see it")
plt.axis("equal")
plt.show()

## 1. Using scikit-learn's GMM
We will first try to use and understand [scikit-learn's implementation of GMM](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html).

In the next section, we will try to propose our own implementation for the same API.

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
# adapted from https://jakevdp.github.io/PythonDataScienceHandbook/05.12-gaussian-mixtures.html
from matplotlib.patches import Ellipse

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))
        
def plot_gmm(gmm, X, label=True, ax=None):
    '''
    gmm: GMM model with sklearn-compatible API
    X: data to model (np.array, 2D (samples, features))
    '''
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=10, cmap='viridis', zorder=2, alpha=0.5)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2, alpha=0.5)
    ax.axis('equal')
    
    p = gmm.get_params()
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)
    plt.title(f"GMM K: {p['n_components']};  iter: {gmm.n_iter_}; cov_type: {p['covariance_type']}")

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Using the previous `plot_gmm()` function, plot the resulting density estimation obtained with different parameters for `GaussianMixture` model.</b>

</div>

In [ ]:
# FIXME
# gmm = ???
# ...
# plot_gmm(???)

In [ ]:
gmm = GaussianMixture(n_components=5, random_state=2, covariance_type="full")

In [ ]:
plt.figure(figsize=(8,8))
plot_gmm(gmm, X_aniso)

## 2. Reimplement our own GMM
This is a pretty tough part, we will try to focus on reimplementing the math properly.

Under the hood, a Gaussian mixture model is very similar to k-means: it uses an expectation–maximization approach which qualitatively does the following:

1. Choose starting guesses for the location and shape

2. Repeat until converged:
   - E-step: for each point, find weights encoding the probability of membership in each cluster
   - M-step: for each cluster, update its location, normalization, and shape based on all data points, making use of the weights


<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Reimplement you GMM training using EM. We suggest that you follow the advisory instructions below.</b>

</div>

### A. Implement a Gaussian function
The goal is to model the generation of some $\mathbf{X}\ \sim\ \mathcal{N}(\boldsymbol\mu,\, \boldsymbol\Sigma)$, i.e. of an observation randomly drawn from a normal distribution of mean $\boldsymbol\mu$ and variance $\boldsymbol\Sigma$.

In this case, if we note
\begin{equation}
{\mathbf x} = (x_1,\ldots,x_n)
\end{equation}
our multivariate observation, then 
according to Wikipedia's page on [Multivariate normal distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Non-degenerate_case) 
the multivariate normal distribution is said to be "non-degenerate" when the symmetric covariance matrix $\boldsymbol {\Sigma }$ is positive definite (**our case, no negative values**).

In this case the distribution has density:
\begin{equation}
\large
p(\mathbf x | \mathbf\mu, \mathbf\Sigma) = \frac 1 {\sqrt{({2\pi})^n|\Sigma|}} \exp\left(-\frac 1 2 ({\mathbf x}-{\boldsymbol\mu})^\mathrm{T}{\boldsymbol\Sigma}^{-1}({\mathbf x}-{\boldsymbol\mu})\right)
\end{equation}

where $\mathbf {x}$ is a real n-dimensional column vector and $|{\boldsymbol {\Sigma }}|\equiv \det {\boldsymbol {\Sigma }}$ is the determinant of ${\boldsymbol {\Sigma }}$.
The quantity ${\sqrt {({\mathbf {x} }-{\boldsymbol {\mu }})^{\mathrm {T} }{\boldsymbol {\Sigma }}^{-1}({\mathbf {x} }-{\boldsymbol {\mu }})}}$ is known as the Mahalanobis distance, which represents the distance of the test point $\mathbf {x}$ from the mean $\boldsymbol {\mu }$.


In [ ]:
# TODO implement this "basic" function
# HINTS use np.pi, np.linalg.det, array.T, np.dot, np.linalg.inv

def gaussian_1(x, mu, sigma):
    '''x: vector
    returns the scalar value of the gaussian (mu, sigma) at x.'''
    # ???
    pass


def gaussian_n(X, mu, sigma):
    '''X: matrix of row vectors
    returns the scalar value of the gaussian (mu, sigma) for each point in X.'''
    # print(X.shape, mu.shape, sigma.shape)
    G = np.apply_along_axis(lambda x: gaussian_1(x, mu, sigma), -1, X)
    return G

In [ ]:
# Some tests to check your implementation
x0 = np.array([[0.05, 1.413, 0.212], [0.85, -0.3, 1.11], [11.1, 0.4, 1.5], [0.27, 0.12, 1.44], [88, 12.33, 1.44]])
mu = np.mean(x0, axis=0)
cov = np.dot((x0 - mu).T, x0 - mu) / (x0.shape[0] - 1)

Y = gaussian_n(x0, mu, cov)
Y
# should return
# array([0.00159853, 0.00481869, 0.00276259, 0.0014309 , 0.00143998])

### B. Initialization

To initialize our GMM, we must initialise our parameters $\pi_k$, $\mu_k$, and $\Sigma_k$. In this case, we are going to use the results of KMeans as an initial value for $\mu_k$, set $\pi_k$ to one over the number of clusters and $\Sigma_k$ to the identity matrix. We could also use random numbers for everything, but using a sensible initialisation procedure will help the algorithm achieve better results.

In [ ]:
# TODO complete the code of the "_initialize()" method in the class below.

### C. Expectation step
This is actually the easiest part.


We should now calculate $\gamma(z_{nk})$. We can achieve this by means of the following expression:

\begin{equation}
\large
\gamma{(z_{nk})}=\frac {\pi_k\mathcal N(\mathbf x_n| \mathbf\mu_k, \mathbf\Sigma_k)}{\sum_{j=1}^K\pi_j\mathcal N(\mathbf x_n| \mathbf\mu_j, \mathbf\Sigma_j)}
\end{equation}

Which is the probability of the $k$-th cluster emitting the observation $x$.

We need to compute the value for all samples, for all clusters.

**This is equivalent to predicting the weighted probability for each observations, normalized by the sum of all predictions for each single observation.**

In [ ]:
# TODO complete the code of the "predict_proba()" method in the class below.

### D. Maximization step

Let us now implement the maximization step. Since $\gamma(z_{nk})$ is common to the expressions for $\pi_k$, $\mu_k$ and $\Sigma_k$, we can simply define:

\begin{equation}
\large
N_k=\sum_{n=1}^N\gamma({z_{nk}})
\end{equation}

(Note hat $N_k$ is a vector of $k$ components.)

And then we can calculate the revised parameters by using (closed-form solution to the minimization of the log-likelihood, see point E.):

\begin{equation}
\large
\pi_k^*=\frac {N_k} N
\end{equation}


\begin{equation}
\large
\mu_k^*=\frac 1 {N_k} \sum_{n=1}^N\gamma({z_{nk}})\mathbf x_n
\end{equation}


\begin{equation}
\large
\Sigma_k^*=\frac 1 {N_k} \sum_{n=1}^N\gamma({z_{nk}})(\mathbf x_n-\mathbf\mu_k)(\mathbf x_n-\mathbf\mu_k)^T
\end{equation}

Note: To calculate the covariance, we define an auxiliary variable __diff__ that contains $(x_n-\mu_k)^T$.

In [ ]:
# TODO complete the code of the "_maximization_step()" method in the class below.

### E. Convergence and Log-likelihood

Let us now determine the [log-likelihood](https://en.wikipedia.org/wiki/Likelihood_function#Log-likelihood) of the model. It is given by:

\begin{equation}
\large
\ln p(\mathbf X)=\sum_{n=1}^N\ln\sum_{k=1}^K\pi_k\mathcal N(\mathbf x_n|\mu_k,\Sigma_k)
\end{equation}

The second summation has already been calculated in the __expectation_step__ function. Let is just make use of it.

Convergence is reached when the difference between the log-likelihood of two consecutive steps decreases below a threshold tolerance value.

In [ ]:
# TODO complete the code of the "get_likelihood()" method in the class below.

In [ ]:
from sklearn.cluster import KMeans  # Needed for initialization

In [ ]:
class MyGMM:
    def __init__(self, n_components, *, random_state=None, max_iter=300, tol=0.0001):
        self.n_components = n_components
        self._random_state = random_state
        self.max_iter = max_iter
        self.tol = tol
        
        # means_array-like of shape (n_components, n_features): The mean of each mixture component.
        self.means_ = None
        # covariances_array-like: The covariance of each mixture component.
        # shape is (n_components, n_features, n_features) if covariance type is 'full'
        self.covariances_ = None
        # weights_: array-like of shape (n_components,): The weights of each mixture components.
        self.weights_ = None

    def _print(self):
        print("="*20)
        print("Weights")
        print(self.weights_)
        print("Means")
        print(self.means_)
        print("Covars")
        print(self.covariances_)

    def fit(self, X):
        self._initialize(X)
        self.n_iter_ = 0
        prev_llh = np.infty
        llh_change = np.infty
        while self.n_iter_ < self.max_iter and llh_change > self.tol:
            print(f"ITER {self.n_iter_}")
            # self._print()
            # Estimation step -----
            gamma_nk, gamma_n = self._expectation_step(X)

            # Maximization step -----
            new_weights, new_means, new_covariances = self._maximization_step(X, gamma_nk)

            # Likelihood -----
            llh, _ = self.get_likelihood(X, gamma_n)
            print(f"log likelihood: {llh:0.4f}")

            # Update parameters -----
            self.weights_ = new_weights
            self.means_ = new_means
            self.covariances_ = new_covariances

            # Update indicators -----
            llh_change = np.abs(prev_llh - llh)
            prev_llh = llh
            self.n_iter_ += 1
        
        return self


    def predict(self, X):
        # Simply pick the best Gaussian for each observation
        return np.argmax(self.predict_proba(X), axis=1)


    def predict_proba(self, X):
        X = np.atleast_2d(X)
        if self.means_ is None:
            raise RuntimeError("You must call fit() before you can predict().")
        probas = np.zeros((X.shape[0], self.n_components), dtype=np.float32)
        for k in range(self.n_components):
            pi_k = self.weights_[k]
            mu_k = self.means_[k]
            sigma_k = self.covariances_[k]
            # FIXME #############################################
            # probas[:,k] = ???
        return probas
    

    def _initialize(self, X):
        num_type = np.float32
        n_features = X.shape[1]
        # FIXME #############################################
        # Use kmeans for first guess. IRL there may be smarter ways!
        # ...
        # (shape: (n_components, n_features))
        # self.means_ = ???
        # and now, how do we init weights and covariances?
        # We pick the simplest possible choice:
        # weights are equals and sum to 1
        # (shape: (n_components, ) )
        # self.weights_ = ???
        # covariance is unit spherical (i.e. Identity(self.n_components))
        # (shape: (n_components, n_features, n_features) )
        # ???
        # self.covariances_ = ???
        # ???
    
    
    def _expectation_step(self, X):
        '''X: shape:(n_samples, n_features) all observations'''
        probas = self.predict_proba(X)
        # normalize by sum of probas for every cluster for each x_i
        denom = np.sum(probas, axis=1)
        probas /= denom.reshape((-1,1))
        return probas, denom  # shape:(n_samples, n_components) and (n_samples,)

    def _maximization_step(self, X, gamma_nk):
        assert X.shape[0] == gamma_nk.shape[0]
        N = gamma_nk.shape[0]
        N_k = np.sum(gamma_nk, axis=0)

        # Compute new weights
        # (share of weights for samples "voting" for each cluster)
        # FIXME #############################################
        # new_weights = ???

        # Compute new means
        # (barycenter of samples weighted by their membership to each cluster)
        # FIXME #############################################
        # new_means = ???

        # Compute new covariances
        # (use new_means! covar of samples weighted by their membership to each cluster)
        # FIXME #############################################
        # new_covariances = ???

        return new_weights, new_means, new_covariances


    def get_likelihood(self, X, gamma_n):
        # FIXME #############################################
        # sample_likelihoods = ???
        return np.sum(sample_likelihoods), sample_likelihoods
    
    def get_params(self):
        return {
            'n_components': self.n_components, 
            'covariance_type': 'full'
            }
        

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Try your GMM implementation on our data, for a couple of cluster values. This will be slow!</b>

</div>

In [ ]:
plt.figure(figsize=(8,8))
plot_gmm(MyGMM(n_components=5, random_state=2), X)

Some interesting questions:

- Can we initialize better?
- Do we need full covariance? How would the computations be impacted by stronger assumptions on the covariances?
- Can we compare the value of the log-likelihood between different trainings? problems? for different numbers of components?

*Hey, let's take some notes here!*

## 3. Select the number of components
There is not no regularization in the algorithm, just bias/capacity tuning based on the number of components.
There are several criterions to select the number of components, always based on how well our model predict the **training** data.

Some criterions:

- [Bayesian information criterion (BIC)](https://en.wikipedia.org/wiki/Bayesian_information_criterion)
- [Akaike information criterion (AIC)](https://en.wikipedia.org/wiki/Akaike_information_criterion)


### BIC
The BIC is formally defined as

$\mathrm {BIC} =k\ln(n)-2\ln({\hat {L}})$


where

- ${\hat {L}}$ = the maximized value of the likelihood function of the model $M$, i.e. ${\hat {L}}=p(x\mid {\widehat {\theta }},M)$, where ${\widehat {\theta }}$ are the parameter values that maximize the likelihood function;
- $x$ = the observed data;
- $n$ = the number of data points in $x$, the number of observations, or equivalently, the sample size;
- $k$ = the number of parameters estimated by the model


### AIC
Let $k$ be the number of estimated parameters in the model. Let ${\hat {L}}$ be the maximum value of the likelihood function for the model. Then the AIC value of the model is the following. 

$\mathrm {AIC} \,=\,2k-2\ln({\hat {L}})$

### What is $\ln({\hat {L}})$?
It is the mean log-probability of each of the samples as predicted by our trained model.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>You can compute the BIC and the AIC directly from a `GaussianMixture` object from scikit-learn. Try to plot the values for these criterions for a range of Gaussians (number of clusters).</b>

*Hint:* you can reuse the code from https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_selection.html
    
</div>



<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>You can try to recompute these criterions on your own GMM class.</b>

*Hint:* check scikit-learn's code!
</div>



# Job done!
Wow, that was a tough one! Congratulations to you!